## установка

In [ ]:
pip install trl

In [ ]:
pip install peft

In [ ]:
pip install bitsandbytes

In [ ]:
pip install accelerate

In [ ]:
pip install datasets

In [ ]:
pip install -U transformers

# Формирование датасета

In [7]:
import random
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [8]:
city_pref = ['', 'г', 'гор', 'г.', 'гор.', 'город']
street_pref = ['', 'ул', 'ул.', 'улица']
house_pref = ['', 'д', 'д.', 'дом']
office_pref = ['', 'кв', 'кв.', 'квартира']

In [9]:
city = ['Москва', 'Санкт-Петербург', 'Волгоград', 'Екатеринбург', 'Хабаровск', 'Владивосток']
country = ['Россия', 'Российская Федерация', 'РФ']
street = ['Люсиновская', 'Маркова', 'Попова', 'Ашихина', 'Гвардейская']
surname_f = ['Иванова', 'Михайлова', 'Сидорова', 'Федорова', 'Шумова']
name_f = ['Светлана', 'Елена', 'Людмила', 'Дарья', 'Лариса', 'Мария']
secondname_f = ['Владимировна', 'Вячеславовна', 'Степановна', 'Сергеевна']
surname_m = ['Иванов', 'Михайлов', 'Сидоров', 'Федоров', 'Шумов']
name_m = ['Сергей', 'Егор', 'Михаил', 'Даниил', 'Владимир', 'Всеволод']
secondname_m = ['Владимирович', 'Вячеславович', 'Степанович', 'Сергеевич']

In [10]:
sentences_list, answers_list = [], []
for i in range(0, 100):
    surname_f_ = surname_f[random.randint(0, len(surname_f)-1)]
    name_f_ = name_f[random.randint(0, len(name_f)-1)]
    secondname_f_ = secondname_f[random.randint(0, len(secondname_f)-1)]
    surname_m_ = surname_m[random.randint(0, len(surname_m)-1)]
    name_m_ = name_m[random.randint(0, len(name_m)-1)]
    secondname_m_ = secondname_m[random.randint(0, len(secondname_m)-1)]
    city_ = city[random.randint(0, len(city)-1)]
    country_ = country[random.randint(0, len(country)-1)]
    street_ = street[random.randint(0, len(street)-1)]
    city_pref_ = city_pref[random.randint(0, len(city_pref)-1)]
    street_pref_ = street_pref[random.randint(0, len(street_pref)-1)]
    house_pref_ = house_pref[random.randint(0, len(house_pref)-1)]
    office_pref_ = office_pref[random.randint(0, len(office_pref)-1)]

    street_ = f'{street_pref_} {street_}'.strip()
    city_ = f'{city_pref_} {city_}'.strip()
    house_ = f'{house_pref_} {random.randint(1, 100)}'.strip()
    office_ = f'{office_pref_} {random.randint(1, 100)}'.strip()
    sentence = f'{surname_f_} {name_f_} {secondname_f_}, {country_}, {city_}, {street_}, {house_}, {office_}'
    sentences_list.append(sentence.replace('  ', ' '))
    answer = f'фамилия: {surname_f_},\nимя: {name_f_},\nотчество: {secondname_f_},\nгород: {city_},\nулица: {street_},\nдом: {house_},\nквартира: {office_}'
    answers_list.append(answer)
    sentence = f'{surname_m_} {name_m_} {secondname_m_}, {country_}, {city_}, {street_}, {house_}, {office_}'
    sentences_list.append(sentence.replace('  ', ' '))
    answer = f'фамилия: {surname_m_},\nимя: {name_m_},\nотчество: {secondname_m_},\nгород: {city_},\nулица: {street_},\nдом: {house_},\nквартира: {office_}'
    answers_list.append(answer)

In [11]:
print(sentences_list[0])
print(answers_list[0])

И
фамилия: Михайлова,
имя: Лариса,
отчество: Степановна,
город: г Екатеринбург,
улица: ул Маркова,
дом: д. 8,
квартира: кв. 100


In [12]:
sentences_list_train, sentences_list_eval, answers_list_train, answers_list_eval = train_test_split(sentences_list, answers_list, test_size=0.2)
dataset_train = {
    'sentence': sentences_list_train,
    'answer': answers_list_train
}
dataset_eval = {
    'sentence': sentences_list_eval,
    'answer': answers_list_eval
}
dataset_hf_1_train = Dataset.from_dict(dataset_train)
dataset_hf_1_eval = Dataset.from_dict(dataset_eval)
dataset_hf_1_train, dataset_hf_1_eval

(Dataset({
     features: ['sentence', 'answer'],
     num_rows: 160
 }),
 Dataset({
     features: ['sentence', 'answer'],
     num_rows: 40
 }))

In [13]:
dataset_hf_2_train = DatasetDict({'train': dataset_hf_1_train, 'val': dataset_hf_1_eval})
dataset_hf_2_train

DatasetDict({
    train: Dataset({
        features: ['sentence', 'answer'],
        num_rows: 160
    })
    val: Dataset({
        features: ['sentence', 'answer'],
        num_rows: 40
    })
})

In [14]:
def format_instruction_for_train(sample):
    return f"""Ты специалист по задаче извлечения сущностей - named entity recognition.
    Ты извлекаешь из предложения все сущности, которые есть в списке сущностей.
    Сущностей, которых нет в этом списке, ты не извлекаешь.
    Ответы ты даешь на русском языке
        ### Список сущностей:
        фамилия, имя, отчество, страна, город, улица, дом, квартира

        ### Предложение:
        {sample["sentence"]}

        ### Ответ:
        {sample["answer"]}
    """

# Модель

## подготовка модели

In [15]:
import torch
from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

In [16]:
from transformers.utils.import_utils import is_accelerate_available, is_bitsandbytes_available
is_accelerate_available(), is_bitsandbytes_available()

(True, True)

In [17]:
bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

In [18]:
model_id = 'mistralai/Mistral-7B-v0.1'

# load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map={'': torch.cuda.current_device()})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [19]:
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [20]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

In [21]:
# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [22]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")


# print the number of trainable model params
print_trainable_parameters(model)

trainable params: 85041152 || all params: 3837112320 || trainable%: 2.2162799758751914


## обучение модели

In [23]:
model_args = TrainingArguments(
    output_dir="mistral-7-ner",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    do_train=True,
    do_eval=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    # bf16=True,
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

In [24]:
max_seq_length = 2048
dataset_trl_train = ConstantLengthDataset(
    dataset=dataset_hf_1_train,
    seq_length=max_seq_length,
    tokenizer=tokenizer,
    formatting_func=format_instruction_for_train,
)

max_seq_length = 2048
dataset_trl_eval = ConstantLengthDataset(
    dataset=dataset_hf_1_eval,
    seq_length=max_seq_length,
    tokenizer=tokenizer,
    formatting_func=format_instruction_for_train,
)

In [25]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_trl_train,
    eval_dataset=dataset_trl_eval,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    packing=True,
    args=model_args,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
# train
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.192200
20,0.107900
30,0.097400


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in

TrainOutput(global_step=30, training_loss=0.13248257637023925, metrics={'train_runtime': 1817.4061, 'train_samples_per_second': 0.264, 'train_steps_per_second': 0.132, 'total_flos': 5305246616125440.0, 'train_loss': 0.13248257637023925, 'epoch': 2.12})

## инференс модели

In [61]:
torch.cuda.empty_cache()

In [27]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

In [28]:
# дообученная модель
model_id = "mistral-7-ner/checkpoint-30"

# load base LLM model, LoRA params and tokenizer
model_fine_tuned = AutoPeftModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
def format_instruction_for_inference(sample):
    return f"""Ты специалист по задаче извлечения сущностей - named entity recognition.
    Ты извлекаешь из предложения все сущности, которые есть в списке сущностей.
    Сущностей, которых нет в этом списке, ты не извлекаешь.
    Ответы ты даешь на русском языке
        ### Список сущностей:
        фамилия, имя, отчество, страна, город, улица, дом, квартира

        ### Предложение:
        {sample["sentence"]}

        ### Ответ:
    """

In [47]:
def inference(sample_for_inference, model):
    prompt = format_instruction_for_inference(sample_for_inference)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # tokenize input text
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to(device)

    # inference, 5 outfit combinations make up around 700-750 tokens
    with torch.inference_mode():
        outputs = model.generate(
          input_ids=input_ids,
          max_new_tokens=800,
          do_sample=True,
          top_p=0.9,
          temperature=0.9
        )
    outputs = outputs.detach().cpu().numpy()
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # outputs is a list of length num_prompts
    # parse the completed part
    output = outputs[0][len(prompt):]
    return output

In [57]:
sample_for_inference_1 = {
    'sentence': 'Федоров Виталий Сергеевич, Россия, г. Екатеринбург, ул. Каховка, 6, кв. 8'
}

### инференс дообученной модели

In [59]:
%%time
res = inference(sample_for_inference=sample_for_inference_1, model=model_fine_tuned)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 11.1 s, sys: 504 ms, total: 11.6 s
Wall time: 13 s


In [60]:
print(res)

 фамилия: Федоров,
имя: Виталий,
отчество: Сергеевич,
город: г. Екатеринбург,
улица: ул. Каховка,
дом: 6,
квартира: кв. 8
    
